In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def filter_odds(span):
    try:
        odd = int(span)
        return True
    except TypeError:
        return False
    except ValueError:
        return False

In [3]:
url = "https://www.actionnetwork.com/soccer/odds"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}

response = requests.get(url, headers=headers)


In [19]:
soup = BeautifulSoup(response.text)
table = soup.tbody
rows = table.find_all("tr")[::2]
odds = [[int(odd.text[1:]) for odd in row.find_all("span") if filter_odds(odd.text)] for row in rows]